# Outline

* Simple hashing - hotel with 26 rooms, labeled A-Z. Guests are assigned to room based on their first name initial. If room occupied guest is not admitted.
* Not a good business model. Lots of demand for room `J` and `M` because John, Mary very popular names. Less demand for `X` or `Z`.
* Superfast to tell if a person is a guest at the hotel. If room corresponding to person's first name initial is not empty, person with that present. $\mathcal O(1)$.
* Many room remain empty. Introduce probing. John goes to `J`. James cannot go to `J` because John is there but what if the next room `K` is available. James can go there. Now, it may take 2 steps to find if James is in hotel. We expect him in `J` but we'll also probe `K`. Still faster than linear search or binary search. 
* Eventually we'll end up underutilizing the hotel.

# Summary of assignments

